<a href="https://colab.research.google.com/github/AirDronzer/keras/blob/master/Handwritten_10_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import

In [0]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image



In [0]:
# Taking MNIST Data Set for Pratice
x='digit_xtrain.csv'
y='digit_ytrain.csv'
x1='digit_xtest.csv'
y1='digit_ytest.csv'


In [0]:
# Reading Data

In [0]:
%%time
xtrain=np.loadtxt(x,dtype=int,delimiter=',')
ytrain=np.loadtxt(y,dtype=int,delimiter=',')
xtest=np.loadtxt(x1,dtype=int,delimiter=',')
ytest=np.loadtxt(y1,dtype=int,delimiter=',')


Wall time: 1min 18s


In [0]:
xtrain.shape

(60000, 784)

In [0]:
#Transforming Data
xtrain = xtrain/255
xtest= xtrain/255

In [0]:
ytrain=np.eye(10)[ytrain]


In [0]:
#Creating Validation Data Set
xval=xtrain[:10000]
yval=ytrain[:10000]
xtrain=xtrain[10000:]
ytrain=ytrain[10000:]

In [0]:
#Setting the graph using tensor placeholder
pl1=tf.placeholder(tf.float32,shape=(None,784))
pl2=tf.placeholder(tf.float32,shape=(None,10))

In [0]:
#Nueral Networks 
int_w1 = tf.truncated_normal(shape=(784,512),stddev=0.1)
w1=tf.Variable(int_w1)
int_b1= tf.constant(shape=[512],value=0.0)
b1=tf.Variable(int_b1)
layer1_in=tf.matmul(pl1,w1)+b1
layer1_out=tf.nn.relu(layer1_in)
# Created First Layer of 512 nodes

In [0]:
#Creating Second Layer
int_w2 = tf.truncated_normal(shape=(512,64),stddev=0.1)
w2=tf.Variable(int_w2)
int_b2= tf.constant(shape=[64],value=0.0)
b2=tf.Variable(int_b2)
layer2_in=tf.matmul(layer1_out,w2)+b2
layer2_out=tf.nn.relu(layer2_in)

In [0]:
# Creating Final Layer of 10 nodes
int_w3 = tf.truncated_normal(shape=(64,10),stddev=0.1)
w3=tf.Variable(int_w3)
int_b3= tf.constant(shape=[10],value=0.0)
b3=tf.Variable(int_b3)
layer3_in=tf.matmul(layer2_out,w3)+b3
layer3_out=tf.nn.softmax(layer3_in)

In [0]:
#Defining Loss and Optimizer
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=pl2,
logits=layer3_out))
optimizer = tf.train.AdamOptimizer(0.001)
minimizer = optimizer.minimize(loss)


In [0]:
#Defining Metrics
predict=tf.equal(tf.argmax(layer3_out,axis=1),
                 tf.argmax(pl2,axis=1))
acc=tf.reduce_mean(tf.cast(predict,tf.float32))

In [0]:
# Creating and running sessions
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)

In [0]:
#Batching the data in 1000
bat_size=1000
num_ex=ytrain.shape[0]
num_it=int(num_ex/bat_size)
flag=0



In [0]:
def next_bat(size,data,labels):
    global num_ex
    global flag
    start=flag
    flag+=size
    end=flag
    if flag>num_ex:
        start=0
        flag=size
    return data[start:end],labels[start:end]

    

In [0]:
# Training the Data
epoch=5
for n in range(epoch):
    for i in range(num_it):
        bat_x,bat_y=next_bat(bat_size,xtrain,ytrain)
        feed_dct={pl1:bat_x,pl2:bat_y}
        sess.run(minimizer,feed_dict=feed_dct)
        bat_acc=sess.run(fetches=[acc],feed_dict={pl1:bat_x,pl2:bat_y})
    print(f'Epoch {n} /t | Traning Accuracy {bat_acc}')
print('Traning Completed')   


Epoch 0 /t | Traning Accuracy [0.955]
Epoch 1 /t | Traning Accuracy [0.964]
Epoch 2 /t | Traning Accuracy [0.971]
Epoch 3 /t | Traning Accuracy [0.973]
Epoch 4 /t | Traning Accuracy [0.976]
Traning Completed


In [0]:

img=Image.open('test_img.png')

In [0]:
test_im=img.convert('L')
test_ima=np.invert(test_im)


In [0]:
test_ima.shape

(28, 28)

In [0]:
test_img=test_ima.ravel()


In [0]:
## Prediction for Our Model
prediction=sess.run(fetches=tf.argmax(layer3_out,axis=1),feed_dict={pl1:[test_img]})
print(f'Prediction for Image is {prediction}')
 

Prediction for Image is [2]
